In [ ]:
import bisect
import copy
import datetime
from datetime import timedelta
import hatyan
import math
import numpy as np
import os
import pandas as pd
import re
from sklearn.decomposition import PCA
import scipy as sc
import scipy.signal
from shapely.ops import transform
from shapely.geometry import Point,Polygon
import sys
import time
import pickle
import pyproj
import pytz
import xarray as xr

In [ ]:
path = os.getcwd()
path = path.split("\\01_Data_Analysis\\03_Hydrodynamic_data")[0]
save_path = path+"\\03_Simulation\\01_Input_data\\02_Hydrodynamic_data"
hydrodynamic_data_file_path = r'C:\Users\floorbakker\Delft University of Technology\Gijs Hendrickx - OSR-data\Referentie\SDS-SALTI_01jan-31dec_2022_his.nc'

## Functions
Calculation functions

In [ ]:
def astronomical_tide(signal_time, signal_values, time_zone=None, constituent_list = []):
    """ Function: calculates the astronomical tidal water level

        Parameters
        ----------
        signal_time: array containing the timestamps of the series
        signal_values: array containing the y-values of the series (water levels or current velocities)
        time_zone: time zone of the timetstamps of the signal time
        constituent_list: list of constituent strings to be resolved on the water level data (see Hatyan documentation)
        
        :returns: primary current velocity time series as a numpy array
    """
    
    #old_stdout = sys.stdout  # backup current stdout
    #sys.stdout = open(os.devnull, "w")
    time_corrections = [datetime.timedelta(seconds=0)]*len(signal_time)
    if time_zone:
        time_zone = pytz.timezone(time_zone)
        times = np.arange(signal_time[0],signal_time[-1]+np.timedelta64(10,'m'),np.timedelta64(10,'m'))
        times = [datetime.datetime.fromtimestamp((time-np.datetime64('1970-01-01T00:00:00'))/np.timedelta64(1, 's')) for time in times]
        times = [utc_to_local(time,time_zone) for time in times]
        times = pd.DatetimeIndex([times])[0]
        time_corrections = [t1.replace(tzinfo=None)-t2 for t1,t2 in zip(times,signal_time)]
    signal_datetime = [(y- np.datetime64('1970-01-01T00:00:00'))/np.timedelta64(1, 's') for y in signal_time]
    duration = signal_datetime[-1] - signal_datetime[0]
    if constituent_list == []:
        periods = [[datetime.timedelta(minutes=25, hours=12), 'tidalcycle'],
                   [datetime.timedelta(days=1), 'day'],
                   [datetime.timedelta(days=14), 'springneap'],
                   [datetime.timedelta(days=31), 'month'],
                   [datetime.timedelta(days=365/2), 'halfyear'],
                   [datetime.timedelta(days=365), 'year']]
        indexes = [deltatime[0] >= datetime.timedelta(seconds=duration) for deltatime in periods]
        if True not in indexes: index = -1
        elif datetime.timedelta(seconds=duration) < datetime.timedelta(minutes=25, hours=12): return [signal_time, signal_values]
        else: index = indexes.index(True)
        const_list = hatyan.get_const_list_hatyan(periods[index][1])
    else:
        const_list = constituent_list
    ts_meas = pd.DataFrame({'values': signal_values}, index=signal_time)
    comp_frommeas, comp_allyears = hatyan.get_components_from_ts(ts=ts_meas, const_list=const_list,nodalfactors=True, return_allyears=True,fu_alltimes=True)
    ts_prediction = hatyan.prediction(comp=comp_frommeas, nodalfactors=True, xfac=True, fu_alltimes=True,times_ext=signal_time, timestep_min=10)
    #sys.stdout = old_stdout  # reset old stdout
    return [[index.timestamp()-time_corrections[idx].total_seconds() for idx,index in enumerate(ts_prediction.index)], [value for value in ts_prediction['values']]],time_corrections,comp_frommeas

def fixed2bearing(east_velocity, north_velocity, principal_direction):
    """ Function: calculates the velocity components in a reference frame parallel to the principal direction of the
                  velocity cluster. It returns the decomposed current velocity magnitudes in a reference frame relative
                  to the given principal direction.

        Parameters
        ----------
        east_velocity: velocity in Eastern direction in m/s (float)
        north_velocity: velocity in Northern direction in m/s (float)
        principal_direction: principal direction in degrees in fixed reference frame North-East (float)
        
        :returns: primary current velocity time series as a numpy array
    """

    bearing_rad = np.radians(principal_direction)
    x_velocity = np.cos(bearing_rad) * east_velocity + np.sin(bearing_rad) * north_velocity
    #y_velocity = -1 * np.sin(bearing_rad) * east_velocity + np.cos(bearing_rad) * north_velocity

    return x_velocity

def fixed2principal_components(east_velocity_list, north_velocity_list):
    """ Function: calculates the principal components from a cluster of velocities. It returns principal direction
                  in degrees (float)

        Parameters
        ----------
        east_velocity_list: list of velocities in Eastern direction (list)
        north_velocity_list: list of velocities in Northern direction (list)
        
        :returns: principal direction in radians as a float
    """

    pca = PCA(n_components=2)
    X = np.column_stack((east_velocity_list, north_velocity_list))
    X_pca = pca.fit(X)
    y_pca = pca.components_[:, 1][0]
    x_pca = pca.components_[:, 0][0]
    theta = np.arctan2(y_pca, x_pca)
    alpha = np.degrees(theta)

    # Correction for positive alpha in coordinate system with positive x-direction upestuary
    if alpha >= 0:
        alpha = alpha - 180  # in degrees
    return alpha

def lag_finder(time_signal, signal1, signal2):
    """ Function: calculates the lag between two signals, for example, to calculate the lag between the vertical and 
                  horizontal tide

        Parameters
        ----------
        time_signal: common time-values as an array
        signal1: y-values of first signal as an array
        signal2: y-values of second signal as an array
        
        :returns: lag as a time delay
    """
    
    #Shifts signals in time to maximize the correlation (signals should have the same timestamps)
    nsamples = len(signal2)

    #Smooth signal
    b, a = sc.signal.butter(2, 0.1)
    signal1 = sc.signal.filtfilt(b, a, signal1)
    signal2 = sc.signal.filtfilt(b, a, signal2)

    #Regularize signals by subtracting mean and dividing by standard deviation
    signal2 -= np.mean(signal2);
    signal2 /= np.std(signal2)
    signal1 -= np.mean(signal1);
    signal1 /= np.std(signal1)

    #Find cross-correlation
    xcorr = sc.signal.correlate(signal2, signal1)

    #Create list of tested time shifts
    dt = np.arange(1 - nsamples, nsamples)

    #Determine time step of the signals
    time_step = time_signal[1] - time_signal[0]

    #Determine phase shift in seconds (when correlation is maximum): delay of signal1 with respect to signal2 (so if signal1 is shifted with delay, there is no phase shift between signals)
    delay = -1 * (dt[xcorr.argmax()] * time_step)

    return delay

def tidal_periods(dataset,horizontal_tidal_period=False,time_zone=None):
    """ Function: calculates the tidal periods

        Parameters
        ----------
        dataset: hydrodynamic data in a xarray DataSet, with following columns: 'Primary current velocity', 
                 'Current velocity', 'STATION', 'Water level', 'TIME'
        horizontal_tidal_period: if a horizontal tidal period should be calculated
        time_zone: time zone of the timestamps in the dataset
        
        Returns
        -------
        times_vertical_tidal_periods: array of vertical tidal periods consisting of list of times and tidal period names
        times_horizontal_tidal_periods: array of horizontal tidal periods consisting of list of times and tidal period names
        phase_lag: phase lags between vertical and horizontal tides
        metadata: metadata consisting of the roots (zero-crossings) of the water level data and velocity data, and phase
                  lag in radians
        astro_water_level_data: astronomic water levels
        primary_current_velocity: primary current velocities
    """
    
    times_vertical_tidal_periods = []
    times_horizontal_tidal_periods = []
    observed_times_horizontal_tidal_periods = []
    phase_lags = []
    primary_current_velocity = []
    time_step = (dataset['TIME'].values[1]-dataset['TIME'].values[0])/np.timedelta64(1,'s')
    if horizontal_tidal_period:
        primary_current_velocity = dataset['Primary current velocity']
        depth_averaged_primary_current_velocity = (dataset['Primary current velocity'].transpose('STATION','TIME','LAYER') * layers).sum('LAYER') 
    metadata = {}
    astro_water_level_data = []
    
    for station in range(len(dataset['STATION'].values)):
        roots_wl = []
        roots_cv = []
        astro_wlev = False
        astro_cvel = False
        if np.max(dataset['Water level'][station].values) > 0.25:
            astro_data_water_level_reduced,_,comp_wlev_red = astronomical_tide(dataset['TIME'].values,dataset['Water level'][station].values,time_zone)
            astro_wlev = True
            astro_water_level_data.append(astro_data_water_level_reduced[1])
        else:
            astro_water_level_data.append(np.zeros(len(dataset['TIME'].values)))
        
        if horizontal_tidal_period and np.max(dataset['Current velocity'][station].values) > 0.25:
            astro_data_primary_current_velocity_reduced,_,comp_cur_red = astronomical_tide(dataset['TIME'].values,depth_averaged_primary_current_velocity[station].values,time_zone,reduced_selected_const)   
            astro_cvel = True
                    
        #High low tidal periods
        if astro_wlev:
            index_prev_root = 0
            roots = sc.interpolate.CubicSpline(astro_data_water_level_reduced[0],astro_data_water_level_reduced[1]).roots()
            roots_wl = [root for root in roots if root>=astro_data_water_level_reduced[0][0] and root<=astro_data_water_level_reduced[0][-1]]
            times_vertical_tidal_period = []
            for root in roots_wl:
                index_current_root = bisect.bisect_right(astro_data_water_level_reduced[0], root)-1
                if index_current_root == -1: 
                    index_current_root = index_current_root + 1
                if len(astro_data_water_level_reduced[1][index_prev_root:index_current_root]) == 0:
                    continue
                wlev_diff_cross = astro_data_water_level_reduced[1][index_current_root+1]-astro_data_water_level_reduced[1][index_current_root-1]
                root = pd.to_datetime(datetime.datetime.fromtimestamp(root,tz=pytz.utc), utc=True).to_datetime64()
                if wlev_diff_cross >= 0:
                    min_wlev = np.min(astro_data_water_level_reduced[1][index_prev_root:index_current_root])
                    times_vertical_tidal_period.append([str(root),'Rising Start'])
                    index_prev_root = index_current_root
                elif wlev_diff_cross <= 0:
                    max_wlev = np.max(astro_data_water_level_reduced[1][index_prev_root:index_current_root])
                    times_vertical_tidal_period.append([str(root),'Falling Start'])
                    index_prev_root = index_current_root

            if times_vertical_tidal_period:
                if times_vertical_tidal_period[0][1] == 'Low water Start': 
                    times_vertical_tidal_period.insert(0,[str(dataset.TIME.values[0]),'High water Start'])
                elif times_vertical_tidal_period[0][1] == 'High water Start': 
                    times_vertical_tidal_period.insert(0,[str(dataset.TIME.values[0]),'Low water Start'])   
                if times_vertical_tidal_period[-1][1] == 'Low water Start': 
                    times_vertical_tidal_period.append([str(dataset.TIME.values[-1]),'High water Start'])
                elif times_vertical_tidal_period[-1][1] == 'High water Start': 
                    times_vertical_tidal_period.append([str(dataset.TIME.values[-1]),'Low water Start']) 
        
        else:
            times_vertical_tidal_period = []
            
        times_vertical_tidal_periods.append(times_vertical_tidal_period)
        
        
        #Rising falling tidal period
        if horizontal_tidal_period and astro_wlev:
            index_prev_root = 0
            roots = sc.interpolate.CubicSpline(astro_data_water_level_reduced[0],astro_data_water_level_reduced[1]).roots()
            roots_wl = [root for root in roots if root>=astro_data_water_level_reduced[0][0] and root<=astro_data_water_level_reduced[0][-1]]
            times_vertical_tidal_period = []
            for root in roots_wl:
                index_current_root = bisect.bisect_right(astro_data_water_level_reduced[0], root)-1
                if index_current_root == -1: 
                    index_current_root = index_current_root + 1
                if len(astro_data_water_level_reduced[1][index_prev_root:index_current_root]) == 0:
                    continue
                wlev_diff_cross = astro_data_water_level_reduced[1][index_current_root+1]-astro_data_water_level_reduced[1][index_current_root-1]
                root = pd.to_datetime(datetime.datetime.fromtimestamp(root,tz=pytz.utc), utc=True).to_datetime64()
                if wlev_diff_cross >= 0:
                    min_wlev = np.min(astro_data_water_level_reduced[1][index_prev_root:index_current_root])
                    rising_start_index = list(astro_data_water_level_reduced[1][index_prev_root:index_current_root]).index(min_wlev)
                    rising_start = astro_data_water_level_reduced[0][index_prev_root:index_current_root][rising_start_index]
                    rising_start = pd.to_datetime(datetime.datetime.fromtimestamp(rising_start,tz=pytz.utc), utc=True).to_datetime64()
                    times_vertical_tidal_period.append([str(rising_start),'Rising Start'])
                    index_prev_root = index_current_root
                elif wlev_diff_cross <= 0:
                    max_wlev = np.max(astro_data_water_level_reduced[1][index_prev_root:index_current_root])
                    falling_start_index = list(astro_data_water_level_reduced[1][index_prev_root:index_current_root]).index(max_wlev)
                    falling_start = astro_data_water_level_reduced[0][index_prev_root:index_current_root][falling_start_index]
                    falling_start = pd.to_datetime(datetime.datetime.fromtimestamp(falling_start,tz=pytz.utc), utc=True).to_datetime64()
                    times_vertical_tidal_period.append([str(falling_start),'Falling Start'])
                    index_prev_root = index_current_root

            if times_vertical_tidal_period:
                if times_vertical_tidal_period[0][1] == 'Falling Start': 
                    times_vertical_tidal_period.insert(0,[str(dataset.TIME.values[0]),'Rising Start'])
                elif times_vertical_tidal_period[0][1] == 'Rising Start': 
                    times_vertical_tidal_period.insert(0,[str(dataset.TIME.values[0]),'Falling Start'])   
                if times_vertical_tidal_period[-1][1] == 'Falling Start': 
                    times_vertical_tidal_period.append([str(dataset.TIME.values[-1]),'Rising Start'])
                elif times_vertical_tidal_period[-1][1] == 'Rising Start': 
                    times_vertical_tidal_period.append([str(dataset.TIME.values[-1]),'Falling Start']) 
            times_vertical_tidal_periods.append(times_vertical_tidal_period)

        #Phase shift
        if horizontal_tidal_period and astro_wlev and astro_cvel:
            phase_lag = lag_finder(dataset['TIME'].values,astro_data_water_level_reduced[1],astro_data_primary_current_velocity_reduced[1])
            phase_lag = phase_lag.astype(dtype='timedelta64[s]')/np.timedelta64(1,'s')
            mean_tidal_period = (dataset['TIME'].values[-1]-dataset['TIME'].values[0])/(len(times_vertical_tidal_period)/2)
            mean_tidal_period = mean_tidal_period.astype(dtype='timedelta64[s]')/np.timedelta64(1,'s')
        else:
            mean_tidal_period = 12.4206012*60*60
            phase_lag = 0.25*mean_tidal_period
        
        #Horizontal tidal period
        if horizontal_tidal_period and astro_wlev and astro_cvel or math.isnan(phase_lag):
            times_horizontal_tidal_period = []
            max_ampl_wlev = np.max(list(comp_wlev_red['A'][0:6]))+np.max(list(comp_wlev_red['A'][6:28]))+np.max(list(comp_wlev_red['A'][28:34]))+np.max(list(comp_wlev_red['A'][34:]))
            max_ampl_cur = np.max(list(comp_cur_red['A'][0:6]))+np.max(list(comp_cur_red['A'][6:28]))+np.max(list(comp_cur_red['A'][28:34]))+np.max(list(comp_cur_red['A'][34:]))
            mean_tidal_period = 12.4206012*60*60
            if max_ampl_cur < 0.4:
                for period in times_vertical_tidal_period:
                    if period[1] == 'Falling Start':
                        times_horizontal_tidal_period.append([period[0],'Ebb Start'])
                    elif period[1] == 'Rising Start':
                        times_horizontal_tidal_period.append([period[0],'Flood Start'])
                phase_lag = 0.25*mean_tidal_period
                observed_times_horizontal_tidal_period = times_horizontal_tidal_period
            else:
                astro_data_primary_current_velocity_reduced[1] = [value-comp_cur_red['A'][0] for value in astro_data_primary_current_velocity_reduced[1]]
                for iteration in range(2*int(abs(phase_lag/mean_tidal_period))+1):
                    if round(phase_lag/mean_tidal_period,1)*2*np.pi < -0.5*np.pi:
                        primary_current_velocity[station] = [-1*value for value in primary_current_velocity[station].values]
                        depth_averaged_primary_current_velocity[station] = [-1*value for value in depth_averaged_primary_current_velocity[station].values]
                        if phase_lag > 0: phase_lag = 0.5*mean_tidal_period-phase_lag
                        elif phase_lag < 0: phase_lag = 0.5*mean_tidal_period+phase_lag
                    elif round(phase_lag/mean_tidal_period,1)*2*np.pi > 0.5*np.pi:
                        phase_lag = phase_lag-mean_tidal_period
                    elif round(phase_lag/mean_tidal_period,1)*2*np.pi < -np.pi:
                        phase_lag = phase_lag+mean_tidal_period
                    else:
                        break

                if phase_lag < 0:
                    phase_lag = 0

                # Interpolation of the variation of the water level and calculation of the zero crossing times
                index_prev_root = 0
                roots = sc.interpolate.CubicSpline(astro_data_primary_current_velocity_reduced[0],depth_averaged_primary_current_velocity.values[station]).roots()
                roots_cv = [root for root in roots if root>=astro_data_primary_current_velocity_reduced[0][0] and root<=astro_data_primary_current_velocity_reduced[0][-1]]
                for root in roots_cv:
                    index_current_root = bisect.bisect_right(astro_data_water_level_reduced[0], root)-2
                    if index_current_root == -1: 
                        index_current_root = index_current_root + 1
                    if len(astro_data_primary_current_velocity_reduced[1][index_prev_root:index_current_root]) == 0:
                        continue 
                    root = pd.to_datetime(datetime.datetime.fromtimestamp(root,tz=pytz.utc), utc=True).to_datetime64()
                    cvel_diff_cross = depth_averaged_primary_current_velocity.values[station][index_current_root+1]-depth_averaged_primary_current_velocity.values[station][index_current_root-1]
                    if cvel_diff_cross < 0:
                        times_horizontal_tidal_period.append([str(root), 'Ebb Start'])
                        index_prev_root = index_current_root
                    elif cvel_diff_cross > 0:
                        times_horizontal_tidal_period.append([str(root), 'Flood Start'])
                        index_prev_root = index_current_root
                
                if times_horizontal_tidal_period:
                    if times_horizontal_tidal_period[0][1] == 'Ebb Start': 
                        times_horizontal_tidal_period.insert(0,[str(dataset.TIME.values[0]),'Flood Start'])
                    elif times_horizontal_tidal_period[0][1] == 'Flood Start': 
                        times_horizontal_tidal_period.insert(0,[str(dataset.TIME.values[0]),'Ebb Start'])   
                    if times_horizontal_tidal_period[-1][1] == 'Ebb Start': 
                        times_horizontal_tidal_period.append([str(dataset.TIME.values[-1]),'Flood Start'])
                    elif times_horizontal_tidal_period[-1][1] == 'Flood Start': 
                        times_horizontal_tidal_period.append([str(dataset.TIME.values[-1]),'Ebb Start']) 
        
        if horizontal_tidal_period:
            phase_lags.append(np.degrees(phase_lag/mean_tidal_period*2*np.pi))
            times_horizontal_tidal_periods.append(times_horizontal_tidal_period)
            metadata[station] = [len(roots_wl),len(roots_cv),np.degrees(phase_lag/mean_tidal_period*2*np.pi)]
        
    return times_vertical_tidal_periods, times_horizontal_tidal_periods, phase_lags, metadata, astro_water_level_data, primary_current_velocity

Derivation functions

In [ ]:
def derive_vertical_tidal_periods(dataset):
    """ Function: initiates the calculation for the vertical tidal periods periods and adds to the dataset

        Parameters
        ----------
        dataset: hydrodynamic data in a xarray DataSet, with following columns: 'Primary current velocity', 
                 'Current velocity', 'STATION', 'Water level', 'TIME'
        horizontal_tidal_period: if a horizontal tidal period should be calculated
        time_zone: time zone of the timestamps in the dataset
        
        :returns: dataset
    """
    
    times_vertical_tidal_periods, times_horizontal_tidal_periods, phase_lags, metadata, astro_wlevs, pcur = tidal_periods(dataset)
    dataset['Astronomic water level'] = xr.DataArray(data=astro_wlevs,dims=["STATION","TIME"],coords={'TIME': ('TIME', dataset['TIME'].values), 'STATION': ('STATION', dataset['STATION'].values)})
    max_length_tidal_period_data = np.max([len(period) for period in times_vertical_tidal_periods])
    tidal_period_data = []
    tidal_periods_data = [[np.zeros(2) for j in np.zeros(max_length_tidal_period_data)] for i in range(len(dataset['STATION']))]

    for data in tidal_periods_data:
        empty_list = []
        for value in data:
            value[:] = np.nan
            empty_list.append(value)
        tidal_period_data.append(empty_list)

    for period in enumerate(times_vertical_tidal_periods):
        for value in enumerate(period[1]):
            tidal_periods_data[period[0]][value[0]] = value[1]

    times_vertical_tidal_periods = tidal_periods_data
    times_vertical_tidal_period_data = xr.DataArray(data=times_vertical_tidal_periods,
                                                    dims=["STATION","VERTICALTIDES","TIDEINFO"],
                                                    coords={'Stations': ('STATION', dataset['STATION'].values)})

    phase_lag_data = xr.DataArray(data=phase_lags,dims=["STATION"])

    dataset['Vertical tidal periods'] = times_vertical_tidal_period_data
    return dataset

def derive_current_velocity_data(dataset):
    """ Function: calculates the current velocity and direction data based on decomposed current velocity data and 
                  adds to the dataset

        Parameters
        ----------
        dataset: hydrodynamic data in a xarray DataSet, with following columns: 'Easting current velocity' and
                 'Northing current velocity'
        
        :returns: dataset
    """
    
    dataset['Current velocity'] = xr.concat([np.sqrt(dataset['Easting current velocity'][index]**2+dataset['Northing current velocity'][index]**2) for index in range(len(dataset['STATION']))],'STATION')
    dataset['Current direction'] = xr.concat([np.degrees(np.arctan2(dataset['Easting current velocity'][index],dataset['Northing current velocity'][index])) for index in range(len(dataset['STATION']))],'STATION')
    return dataset

def derive_primary_current_velocity(dataset):
    """ Function: initates the calculation of the primary current velocity and adds it to the dataset
    
        Parameters
        ----------
        dataset: hydrodynamic data in a xarray DataSet, with following columns: 'Easting current velocity' and
                 'Northing current velocity'
        
        :returns: dataset
    """
    
    depth_averaged_easting_current_velocity_data = (dataset['Easting current velocity'].transpose('STATION','TIME','LAYER') * layers).sum('LAYER')
    depth_averaged_northing_current_velocity_data = (dataset['Northing current velocity'].transpose('STATION','TIME','LAYER') * layers).sum('LAYER')
    depth_averaged_principle_components = []
    primary_current_velocity = []
    for station in range(len(dataset['STATION'].values)):
        depth_averaged_principle_components.append(fixed2principal_components(depth_averaged_easting_current_velocity_data[station],
                                                                              depth_averaged_northing_current_velocity_data[station]))
        
        primary_current_velocity.append(fixed2bearing(dataset['Easting current velocity'][station],
                                                      dataset['Northing current velocity'][station],
                                                      depth_averaged_principle_components[station]))

    dataset['Primary current velocity'] = xr.concat(primary_current_velocity,'STATION')
    return dataset

def derive_tidal_periods(dataset):   
    """ Function: initates the calculation of the horizontal and vertical tidal periods and adds it to the dataset
    
        Parameters
        ----------
        dataset: hydrodynamic data in a xarray DataSet, with following columns: 'Easting current velocity' and
                 'Northing current velocity'
        
        :returns: dataset
    """
    
    times_vertical_tidal_periods, times_horizontal_tidal_periods, phase_lags, metadata, astro_wlevs, pcur = tidal_periods(dataset)
    dataset['Primary current velocity'] = pcur
    combined_tidal_periods = [times_vertical_tidal_periods,times_horizontal_tidal_periods]
    for repetition in range(2):
        max_length_tidal_period_data = np.max([len(period) for period in combined_tidal_periods[repetition]])
        tidal_period_data = []
        tidal_periods_data = [[np.zeros(2) for j in np.zeros(max_length_tidal_period_data)] for i in range(len(dataset['STATION']))]
        
        for data in tidal_periods_data:
            empty_list = []
            for value in data:
                value[:] = np.nan
                empty_list.append(value)
            tidal_period_data.append(empty_list)

        for period in enumerate(combined_tidal_periods[repetition]):
            for value in enumerate(period[1]):
                tidal_periods_data[period[0]][value[0]] = value[1]

        combined_tidal_periods[repetition] = tidal_periods_data

    times_vertical_tidal_period_data = xr.DataArray(data=combined_tidal_periods[0],
                                                    dims=["STATION","VERTICALTIDES","TIDEINFO"],
                                                    coords={'Stations': ('STATION', dataset['STATION'].values)})

    times_horizontal_tidal_period_data = xr.DataArray(data=combined_tidal_periods[1],
                                                      dims=["STATION","HORIZONTALTIDES","TIDEINFO"],
                                                      coords={'Stations': ('STATION', dataset['STATION'].values)})

    phase_lag_data = xr.DataArray(data=phase_lags,dims=["STATION"])

    dataset['Vertical tidal periods'] = times_vertical_tidal_period_data
    dataset['Horizontal tidal periods'] = times_horizontal_tidal_period_data
    dataset['Phase lag'] = phase_lag_data
    return dataset

## Load port network

In [ ]:
with open(path+"\\03_Simulation\\01_Input_data\\01_Geospatial_data\\network"+"\\PoR_graph_with_information.pickle", 'rb') as f:
    FG = pickle.load(f)

## Load hydrodynamic dataset

In [ ]:
hydrodynamic_data = xr.open_dataset(hydrodynamic_data_file_path)

### Preprocessing
Change the type of the station names

In [ ]:
hydrodynamic_data['NAMPOL'] = [name.split(' ')[0] for name in hydrodynamic_data['NAMPOL'].values.astype(str)]

Rename coordinates and variables

In [ ]:
hydrodynamic_data = hydrodynamic_data.rename({'NAMPOL':'STATION'})
hydrodynamic_data = hydrodynamic_data.rename({'ZWL':'Water level'})
hydrodynamic_data['ZCURU'] = hydrodynamic_data['ZCURU'].rename({'STATIONCUR':'STATION'})
hydrodynamic_data = hydrodynamic_data.rename({'ZCURU':'Easting current velocity'})
hydrodynamic_data['ZCURV'] = hydrodynamic_data['ZCURV'].rename({'STATIONCUR':'STATION'})
hydrodynamic_data = hydrodynamic_data.rename({'ZCURV':'Northing current velocity'})
hydrodynamic_data = hydrodynamic_data.rename({'ZKPOLC':'Layer depths'})
hydrodynamic_data = hydrodynamic_data.rename({'GRO':'Salinity'})

Add and remove coordinates

In [ ]:
layers = [0.12, 0.12, 0.11, 0.11, 0.11, 0.11, 0.11, 0.09, 0.06, 0.06]
hydrodynamic_data = hydrodynamic_data.assign_coords({'LAYER':layers})
hydrodynamic_data['Salinity'] = hydrodynamic_data['Salinity'].squeeze('CONSTIT')

Add the depths

In [ ]:
depths = []
for station_index,_ in enumerate(hydrodynamic_data['STATION']):
    data = hydrodynamic_data.isel({'TIME':32000}).isel({'STATION':station_index})
    depths.append(np.round(data['Layer depths'].values[-1]-(data['Layer depths'].values[-2]-data['Layer depths'].values[-1])/(layers[-2]/2+layers[-1]/2)*(layers[-1]/2),2))
hydrodynamic_data['Depth'] = xr.DataArray(data=depths,coords={'STATION':hydrodynamic_data['STATION']})

Expand dataset

In [ ]:
text = open(path+"\\00_Input_data\\02_Hydrodynamic_data"+'\\points_NG04_2023_v103_v3_0.obs','r')
stations = {}
node_stations = []
htide_stations = []
for line in text.readlines():
    n = int(re.search(r'(?:[, ])(\d+)\n', line).group(1))
    m = int(re.search(r'(?:[, ]) (\d+) ', line).group(1))
    station = line.split(' ')[0]
    stations[station] = (n,m)
    if 'P' in line[0]:
        node_stations.append((station,stations[station]))
    if 'C' in line[0]:
        htide_stations.append((station,stations[station]))

In [ ]:
surfpath = 'C:\\Users\\floorbakker\\surfdrive\\Documents\\PhD SALTISolutions\\03_Models\\OSR2022\\'
grid = pd.read_csv(surfpath+'NG04_single_2016_v601.csv',delimiter=';',skiprows=2)
grid = grid.rename(columns={'0 0 0':'ETA','Unnamed: 1':0,'Unnamed: 2':1,'Unnamed: 3':2,'Unnamed: 4':3,'Unnamed: 5':4,})
grid = grid.set_index('ETA')
grid.replace(0, np.nan, inplace=True)
separation_locs = []
for loc,index in enumerate(grid.index):
    if 'ETA=' in str(index):
        separation_locs.append(loc)
separation_locs.append(len(grid))
x = []
y = []
for loc1,loc2 in zip(separation_locs[:-1],separation_locs[1:]):
    subgrid = grid.iloc[loc1:loc2]
    if loc1 < len(grid)/2:
        x.append(subgrid.to_numpy().flatten())
    else:
        y.append(subgrid.to_numpy().flatten())
x = xr.DataArray(x,dims=['N','M'],coords={'N':np.arange(1,len(x)+1,1),'M':np.arange(1,len(x[0])+1,1)})
y = xr.DataArray(y,dims=['N','M'],coords={'N':np.arange(1,len(x)+1,1),'M':np.arange(1,len(x[0])+1,1)})  
griddata = xr.Dataset()
griddata['x'] = x
griddata['y'] = y

In [ ]:
missing_nodes = []
for node in FG.nodes:
    for index,(node_name,(n,m)) in enumerate(node_stations):
        if node_name.split('P_')[-1] == node:
            break
        if index+1 == len(node_stations):
            missing_nodes.append(node)

In [ ]:
utm = pyproj.CRS('EPSG:28992')
wgs84 = pyproj.CRS('EPSG:4326')
wgs_to_utm = pyproj.Transformer.from_crs(wgs84,utm,always_xy=True).transform
utm_to_wgs = pyproj.Transformer.from_crs(utm,wgs84,always_xy=True).transform

In [ ]:
offset = 400
selected_gridcell = {}
for missing_node in missing_nodes:
    selected_gridcell[missing_node] = np.NaN
    rd_geometry = transform(wgs_to_utm, FG.nodes[missing_node]['geometry'])
    subset = griddata.where((griddata.x > rd_geometry.x-offset) & (griddata.x < rd_geometry.x+offset) &
                            (griddata.y > rd_geometry.y-offset) & (griddata.y < rd_geometry.y+offset))
    stacked_subset_x = subset.x.stack(x=['N','M'])
    stacked_subset_y = subset.y.stack(x=['N','M'])
    stacked_subset_x_notna = stacked_subset_x[stacked_subset_x.notnull()]
    stacked_subset_y_notna = stacked_subset_y[stacked_subset_y.notnull()]
    subset_x = stacked_subset_x_notna.unstack()
    subset_y = stacked_subset_y_notna.unstack()
    subset = xr.Dataset()
    subset['x'] = subset_x
    subset['y'] = subset_y
    subset = subset.sortby(['N','M'])
    if not subset.N.values.size or not subset.M.values.size:
        continue
    
    subset_n_values = np.arange(np.min(subset.N.values)-1,np.max(subset.N.values))
    subset_m_values = np.arange(np.min(subset.M.values)-1,np.max(subset.M.values))
    for n1,n2 in zip(subset_n_values[:-1],subset_n_values[1:]):
        for m1,m2 in zip(subset_m_values[:-1],subset_m_values[1:]):
            (x1,y1) = (griddata.x.values[n1][m1],griddata.y.values[n1][m1])
            (x2,y2) = (griddata.x.values[n1][m2],griddata.y.values[n1][m2])
            (x3,y3) = (griddata.x.values[n2][m2],griddata.y.values[n2][m2])
            (x4,y4) = (griddata.x.values[n2][m1],griddata.y.values[n2][m1])
            if not (True if True in np.isnan(np.array([x1,x2,x3,x4,y1,y2,y3,y4])) else False):
                polygon = Polygon([Point(x1,y1),Point(x2,y2),Point(x3,y3),Point(x4,y4)])
                if polygon.contains(rd_geometry):
                    selected_gridcell[missing_node] = (n2+1,m2+1)
                    break
    
    if not isinstance(selected_gridcell[missing_node],float):
        continue
    
    distance_to_cell = np.NaN*np.ones((len(griddata.N.values),len(griddata.M.values)))
    for n1,n2 in zip(subset_n_values[:-1],subset_n_values[1:]):
        for m1,m2 in zip(subset_m_values[:-1],subset_m_values[1:]):
            (x1,y1) = (griddata.x.values[n1][m1],griddata.y.values[n1][m1])
            (x2,y2) = (griddata.x.values[n1][m2],griddata.y.values[n1][m2])
            (x3,y3) = (griddata.x.values[n2][m2],griddata.y.values[n2][m2])
            (x4,y4) = (griddata.x.values[n2][m1],griddata.y.values[n2][m1])
            if not (True if True in np.isnan(np.array([x1,x2,x3,x4,y1,y2,y3,y4])) else False):
                distance_to_cell[n2][m2] = Polygon([Point(x1,y1),Point(x2,y2),Point(x3,y3),Point(x4,y4)]).distance(rd_geometry)
    
    if math.isnan(np.nanmin(distance_to_cell)):
        continue
    n2,m2 = (np.argwhere(distance_to_cell == np.nanmin(distance_to_cell))[0][0]+1,
             np.argwhere(distance_to_cell == np.nanmin(distance_to_cell))[0][1]+1)
    selected_gridcell[missing_node] = (n2,m2)

In [ ]:
replacement_nodes = {}
for missing_node,missing_cell in selected_gridcell.items():
    for station,cell in stations.items():
        if missing_cell == cell:
            replacement_nodes[missing_node] = station
            break
            
    if missing_node not in replacement_nodes.keys():
        replacement_nodes[missing_node] = np.NaN
replacement_nodes['C_Heysehaven'] = 'P_8861973'
replacement_nodes['L_Buitensluis'] = 'P_8861290'
replacement_nodes['L_Monstersche_sluis'] = 'P_8865732'

In [ ]:
dummy_dataset = hydrodynamic_data.isel({'STATION':0})
dummy_dataset = dummy_dataset.where(lambda x: x == 0, 0)

In [ ]:
replacement_dataset = xr.Dataset()
for missing_node,replacement_node in replacement_nodes.items():
    if isinstance(replacement_nodes[missing_node],str):
        replacement_dataset_i = hydrodynamic_data.sel({'STATION':replacement_node})
    else:
        replacement_dataset_i = dummy_dataset.copy()
    replacement_dataset_i = replacement_dataset_i.assign_coords({'STATION':missing_node})
    
    if not len(replacement_dataset.sizes):
        replacement_dataset = replacement_dataset_i
    else:
        replacement_dataset = xr.concat([replacement_dataset,replacement_dataset_i],dim='STATION')
    
replacement_dataset = replacement_dataset.transpose('TIME','STATION','LAYER') 
replacement_dataset['STATION'] = replacement_dataset['STATION'].astype(object)

In [ ]:
coded_names_replacement_data = ['P_'+station if not (station.find('C_')+1 or station.find('L_')+1) else station for station in replacement_dataset.STATION.values]
replacement_dataset = replacement_dataset.assign_coords({'STATION':coded_names_replacement_data})

In [ ]:
station_names = list(hydrodynamic_data['STATION'].values)
station_names.extend(list(replacement_dataset['STATION'].values))

In [ ]:
hydrodynamic_data = hydrodynamic_data.merge(replacement_dataset)

Rename stations

In [ ]:
hydrodynamic_data = hydrodynamic_data.reindex({'STATION':station_names})

In [ ]:
hydrodynamic_data['TIME'] = hydrodynamic_data['TIME'].where(hydrodynamic_data['TIME'] != -9999.)

In [ ]:
hydrodynamic_data.to_netcdf(save_path+'\\hydrodynamic_data_preprocessed.nc')

### Processing

In [ ]:
hydrodynamic_data = xr.open_dataset(save_path+'\\hydrodynamic_data_preprocessed.nc')

In [ ]:
freq_list = hatyan.get_full_const_list_withfreqs()
const_list = hatyan.get_const_list_hatyan('year')
selected_const = []
for const in const_list:
    index = list(freq_list.index).index(const)
    if freq_list['freq'][index] < (1/6):
        selected_const.append(const)
reduced_selected_const = selected_const[3:]

Remove salinity data

In [ ]:
hydrodynamic_data = hydrodynamic_data.drop('Salinity')

#### Water levels
Select data

In [ ]:
wlev_hydrodynamic_data = hydrodynamic_data.sel({'STATION':[station for station in hydrodynamic_data.STATION.values if station.find('P_')+1]})

Replace non-used data

In [ ]:
wlev_hydrodynamic_data = wlev_hydrodynamic_data.rename({'Easting current velocity':'Current velocity'})
wlev_hydrodynamic_data = wlev_hydrodynamic_data.rename({'Northing current velocity':'Current direction'})

In [ ]:
wlev_hydrodynamic_data['Current velocity'] = wlev_hydrodynamic_data['Current velocity'].where(lambda x: x == 0, 0)

In [ ]:
wlev_hydrodynamic_data['Current direction'] = wlev_hydrodynamic_data['Current velocity']

In [ ]:
wlev_hydrodynamic_data['Layer depths'] = wlev_hydrodynamic_data['Layer depths'].where(lambda x: x == 0, 0)

In [ ]:
wlev_hydrodynamic_data = wlev_hydrodynamic_data.transpose('STATION','TIME','LAYER')

In [ ]:
wlev_hydrodynamic_data = derive_vertical_tidal_periods(wlev_hydrodynamic_data)

In [ ]:
wlev_hydrodynamic_data.to_netcdf(save_path+'\\wlev_hydrodynamic_data.nc')

### Current velocities
Calculate current velocities and directions

In [ ]:
current_hydrodynamic_data = hydrodynamic_data.sel({'STATION':[station for station in hydrodynamic_data.STATION.values if station.find('C_')+1]})

In [ ]:
current_hydrodynamic_data = current_hydrodynamic_data.transpose('STATION','TIME','LAYER')

In [ ]:
layers = [0.12, 0.12, 0.11, 0.11, 0.11, 0.11, 0.11, 0.09, 0.06, 0.06]

In [ ]:
current_hydrodynamic_data = derive_primary_current_velocity(current_hydrodynamic_data)

In [ ]:
current_hydrodynamic_data = derive_current_velocity_data(current_hydrodynamic_data)

In [ ]:
current_hydrodynamic_data = derive_tidal_periods(current_hydrodynamic_data)

In [ ]:
current_hydrodynamic_data = current_hydrodynamic_data.drop('Stations')
current_hydrodynamic_data = current_hydrodynamic_data.drop('Easting current velocity')
current_hydrodynamic_data = current_hydrodynamic_data.drop('Northing current velocity')
current_hydrodynamic_data = current_hydrodynamic_data.drop('Phase lag')
final_hydrodynamic_data = final_hydrodynamic_data.drop('Depth')
final_hydrodynamic_data = final_hydrodynamic_data.drop('Layer depths')
final_hydrodynamic_data = final_hydrodynamic_data.drop('Astronomic water level')

In [ ]:
current_hydrodynamic_data.to_netcdf(save_path+'\\current_hydrodynamic_data.nc')

## Combine water level and current velocity data

In [ ]:
wlev_hydrodynamic_data = xr.open_dataset(save_path+'\\wlev_hydrodynamic_data.nc')
current_hydrodynamic_data = xr.open_dataset(save_path+'\\current_hydrodynamic_data.nc')

In [ ]:
wlev_variable_names = list(wlev_hydrodynamic_data.variables.keys())
current_variable_names = list(current_hydrodynamic_data.variables.keys())

In [ ]:
wlev_hydrodynamic_data['Current velocity'] = wlev_hydrodynamic_data['Current velocity'].where(lambda x: x==0,0)
wlev_hydrodynamic_data['Current direction'] = wlev_hydrodynamic_data['Current velocity'].copy()
missing_variables_wlev_data = list(set(current_variable_names)-set(wlev_hydrodynamic_data))
for coord in ['STATION','TIME','LAYER']:
    missing_variables_wlev_data.remove(coord)
for missing_variable in missing_variables_wlev_data:  
    if missing_variable in ['Vertical tidal periods','Horizontal tidal periods']:
        if missing_variable == 'Vertical tidal periods':
            dim2name = 'VERTICALTIDES'
        elif missing_variable == 'Horizontal tidal periods':
            dim2name = 'HORIZONTALTIDES'
        dummy_dataarray_station = current_hydrodynamic_data[missing_variable].isel({'STATION':[0]}).where(lambda x: x == 'nan','nan')
        dummy_dataarray_stations = xr.DataArray([dummy_dataarray_station.values]*316,
                                                coords={'dim_0':wlev_hydrodynamic_data.STATION.values})
        dummy_dataarray_stations = dummy_dataarray_stations.squeeze('dim_1')
        dummy_dataarray_stations = dummy_dataarray_stations.rename({'dim_0':'STATION','dim_2':dim2name,'dim_3':'TIDEINFO'})
        wlev_hydrodynamic_data[missing_variable] = dummy_dataarray_stations
    else:
        wlev_hydrodynamic_data[missing_variable] = wlev_hydrodynamic_data['Current velocity'].copy()

In [ ]:
missing_variables_cur_data = list(set(wlev_hydrodynamic_data)-set(current_variable_names))
current_hydrodynamic_data['Water level'] = current_hydrodynamic_data['Water level'].where(lambda x: x == 0,0)
for missing_variable in missing_variables_cur_data:
    current_hydrodynamic_data[missing_variable] = current_hydrodynamic_data['Water level'].copy()

In [ ]:
wlev_hydrodynamic_data = wlev_hydrodynamic_data.drop('Stations')

In [ ]:
index = np.argmax([len(wlev_hydrodynamic_data.VERTICALTIDES.values),len(current_hydrodynamic_data.VERTICALTIDES.values)])
max_tides = np.max([len(wlev_hydrodynamic_data.VERTICALTIDES.values),len(current_hydrodynamic_data.VERTICALTIDES.values)])
if not index:
    dataset = current_hydrodynamic_data
else:
    dataset = wlev_hydrodynamic_data


stations = dataset.STATION.values
data = np.zeros((len(stations),max_tides,2))
data = np.where(data,'','nan')
data = data.astype(dtype='U32')
for station_index,station in enumerate(stations):
    tidal_data = dataset.sel({'STATION':station})['Vertical tidal periods'].values
    for tide in dataset.VERTICALTIDES.values:
        data[station_index][tide][0] = tidal_data[tide][0]
        data[station_index][tide][1] = tidal_data[tide][1]

dataset = dataset.drop('Vertical tidal periods')
dataset['Vertical tidal periods'] = xr.DataArray(data,coords={'STATION':stations},dims={'STATION':stations,'VERTICALTIDES':np.arange(0,max_tides),'TIDEINFO':np.arange(0,2)})
if not index:
    current_hydrodynamic_data = dataset
else:
    wlev_hydrodynamic_data = dataset

In [ ]:
final_hydrodynamic_data = xr.Dataset()
for variable in list(wlev_hydrodynamic_data.variables.keys())[3:]:
    merged_variable = xr.concat([wlev_hydrodynamic_data[variable],current_hydrodynamic_data[variable]],dim='STATION')
    final_hydrodynamic_data[variable] = merged_variable

In [ ]:
final_hydrodynamic_data = final_hydrodynamic_data.transpose('TIME','STATION','LAYER','HORIZONTALTIDES','VERTICALTIDES','TIDEINFO')

## Rename stations

In [ ]:
final_station_names = []
for station in final_hydrodynamic_data.STATION.values:
    if station.find('P_')+1:
        final_station_names.append(station.split('P_')[-1])
    if station.find('C_')+1:
        final_station_names.append(station.split('C_')[-1])
        
final_hydrodynamic_data = final_hydrodynamic_data.assign_coords({'STATION':final_station_names})

## Add MBLs (based on network)

In [ ]:
with open(path+'\\geospatial_data'+"\\PoR_graph_with_information.pickle", 'rb') as f:
    FG = pickle.load(f)

In [ ]:
MBL_data = np.zeros((len(final_hydrodynamic_data.STATION.values),len(final_hydrodynamic_data.TIME.values)))

In [ ]:
for station_index,node in enumerate(final_hydrodynamic_data.STATION.values):
    if node in list(FG.nodes):
        MBL = FG.nodes[node]['MBL']
        MBL_data[station_index][:] = MBL

In [ ]:
MBL_data = MBL_data.transpose()

In [ ]:
final_hydrodynamic_data['MBL'] = xr.DataArray(MBL_data,{'TIME':final_hydrodynamic_data.TIME.values,'STATION':final_hydrodynamic_data.STATION.values})

In [ ]:
final_hydrodynamic_data.to_netcdf(save_path+'\\hydrodynamic_data_final.nc')